## Notebook Template to Quickly Test Things Out

In [1]:
# General imports
# import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
import pandas as pd

In [3]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists
# from utils.model_utils import prepare_device
# # from parse_config import ConfigParser
# from data_loader import data_loaders
# import model.model as module_arch

In [164]:
# Define constants, paths
config_path = 'configs/'
# timestamp = '0112_121958' # 37 dogs
timestamp = '0112_163516' # 128 cats
csv_path = os.path.join('saved', 'edit', 'trials', 'CINIC10_ImageNet-VGG_16', timestamp, 'results_table.csv')


In [165]:
# Load CSV as pandas dataframe
df = pd.read_csv(csv_path)
n_total = len(df)
print("CSV loaded from {}".format(csv_path))
print("{} rows".format(n_total))

CSV loaded from saved/edit/trials/CINIC10_ImageNet-VGG_16/0112_163516/results_table.csv
128 rows


In [166]:
# Round all numbers to 3 decimal places
df.round(3)
mean_df = df.mean()
std_df = df.std()

In [194]:
def print_summary(df_, metrics=None):
    if metrics == None:
        metrics = [['{} Accuracy', '{} Mean Precision', '{} Mean Recall', '{} Mean F1'], 
               ['{} Target Precision', '{} Target Recall', '{} Target F1'],
               ['{} Orig Pred Precision', '{} Orig Pred Recall', '{} Orig Pred F1']]
    assert type(metrics) == list and type(metrics[0]) == list, "Metrics must be a 2-D list"
    
    mean_df = df_.mean()
    std_df = df_.std()
    print("{:<30} {:<15} {:<20}".format("Metric", "Pre-Edit", "Post-Edit"))
    for row in metrics:
        for metric in row:
            print("{:<30} {:<15.3f} {:.3f}({:.3f})".format(
                metric, mean_df[metric.format("Pre")],
                mean_df[metric.format("Post")], std_df[metric.format("Post")]))
        print("")

In [167]:
metrics = [['{} Accuracy', '{} Mean Precision', '{} Mean Recall', '{} Mean F1'], 
           ['{} Target Precision', '{} Target Recall', '{} Target F1'],
           ['{} Orig Pred Precision', '{} Orig Pred Recall', '{} Orig Pred F1']]
print("{:<30} {:<15} {:<20}".format("Metric", "Pre-Edit", "Post-Edit"))
for row in metrics:
    for metric in row:
        print("{:<30} {:<15.3f} {:.3f}({:.3f})".format(
            metric, mean_df[metric.format("Pre")],
            mean_df[metric.format("Post")], std_df[metric.format("Post")]))
    print("")

Metric                         Pre-Edit        Post-Edit           
{} Accuracy                    0.687           0.642(0.046)
{} Mean Precision              0.692           0.665(0.050)
{} Mean Recall                 0.687           0.642(0.046)
{} Mean F1                     0.684           0.631(0.055)

{} Target Precision            0.554           0.387(0.086)
{} Target Recall               0.549           0.693(0.080)
{} Target F1                   0.551           0.485(0.058)

{} Orig Pred Precision         0.681           0.474(0.409)
{} Orig Pred Recall            0.633           0.258(0.266)
{} Orig Pred F1                0.647           0.309(0.304)



## Hypothesis 1: Masked modifications will have greater changes than noise

In [170]:
masked_rows = df[df['ID'].str.contains('masked')]
n_masked = len(masked_rows)

gaussian_rows = df[df['ID'].str.contains('gaussian')]
n_gaussian = len(gaussian_rows)
# Compare mean post edit accuracy, precision, recall, and f1
mean_masked = masked_rows.mean()
mean_gaussian = gaussian_rows.mean()
std_masked = masked_rows.std()
std_gaussian = gaussian_rows.std()
# print(mean_masked)

metrics = [['{} Accuracy', '{} Mean Precision', '{} Mean Recall', '{} Mean F1'], 
           ['{} Target Precision', '{} Target Recall', '{} Target F1'],
           ['{} Orig Pred Precision', '{} Orig Pred Recall', '{} Orig Pred F1']]
print("{:<30} {:<15} {:<20} {:<20}".format("Metric", "Pre-Edit", "Masked", "Gaussian"))
for row in metrics:
    for metric in row:
        print("{:<30} {:<15.3f} {:.3f}({:.3f}) {:<6} {:.3f}({:.3f})".format(
            metric, mean_masked[metric.format("Pre")],
            mean_masked[metric.format("Post")], std_masked[metric.format("Post")], "",
            mean_gaussian[metric.format("Post")], std_gaussian[metric.format("Post")]))
    print("")

Metric                         Pre-Edit        Masked               Gaussian            
{} Accuracy                    0.687           0.635(0.049)        0.663(0.028)
{} Mean Precision              0.692           0.660(0.052)        0.681(0.038)
{} Mean Recall                 0.687           0.635(0.049)        0.663(0.028)
{} Mean F1                     0.684           0.623(0.058)        0.654(0.039)

{} Target Precision            0.554           0.372(0.085)        0.436(0.070)
{} Target Recall               0.549           0.713(0.079)        0.629(0.039)
{} Target F1                   0.551           0.477(0.060)        0.509(0.047)

{} Orig Pred Precision         0.679           0.421(0.414)        0.649(0.343)
{} Orig Pred Recall            0.638           0.219(0.255)        0.385(0.263)
{} Orig Pred F1                0.648           0.265(0.295)        0.453(0.291)



### Hypothesis 2: How many edits actually improved all three metrics for target class?

Will these also incur larger harm in overall metrics?

In [172]:
improve_target_rows = df[
    (df['Post Target Precision'] > df['Pre Target Precision']) &
    (df['Post Target Recall'] > df ['Pre Target Recall']) & 
    (df['Post Target F1'] > df['Pre Target F1'])]      

improve_target_recall = df[df['Post Target Recall'] > df ['Pre Target Recall']] 
improve_target_f1 = df[df['Post Target F1'] > df ['Pre Target F1']] 
improve_target_precision = df[df['Post Target Precision'] > df ['Pre Target Precision']] 
    
improve_target_precision_f1 = pd.merge(improve_target_precision, improve_target_f1, how='inner', on=['ID'])
improve_target_recall_f1 = pd.merge(improve_target_recall, improve_target_f1, how='inner', on=['ID'])
improve_target_recall_and_precision = pd.merge(improve_target_recall, improve_target_precision, how='inner', on=['ID'])

print("{} edits improved all target metrics ".format(len(improve_target_rows)))
print("{} edits improved target recall".format(len(improve_target_recall)))
print("{} edits improved target F1".format(len(improve_target_f1)))
print("{} edits improved target precision".format(len(improve_target_precision)))
print("{} edits improved target precision + f1".format(len(improve_target_precision_f1)))
print("{} edits improved target recall + f1".format(len(improve_target_recall_f1)))
print("{} edits improved target precision + recall".format(len(improve_target_recall_and_precision)))


0 edits improved all target metrics 
127 edits improved target recall
6 edits improved target F1
0 edits improved target precision
0 edits improved target precision + f1
6 edits improved target recall + f1
0 edits improved target precision + recall


## Hypothesis 3: If the edit improved the target class (let's say F1), then the metrics of the originally predicted class will be worse

In [181]:
 orig_metrics = ['{} Orig Pred Precision', '{} Orig Pred Recall', '{} Orig Pred F1']
mean_improve_target_f1 = improve_target_f1.mean()
std_improve_target_f1 = improve_target_f1.std()

print("Average mean metrics in original predicted class for rows that improved f1 in target class ({} samples)".format(len(improve_target_f1)))
print("{:<30} {:<15} {:<20}".format("Metric", "Pre-Edit", "Post-Edit"))
for metric in orig_metrics:
    print("{:<30} {:.3f}({:.3f}) {:<6} {:.3f}({:.3f})".format(
        metric, 
        mean_improve_target_f1[metric.format("Pre")], 
        std_improve_target_f1[metric.format("Pre")], 
        "",
        mean_improve_target_f1[metric.format("Post")],
        std_improve_target_f1[metric.format("Post")]))
    
not_improve_target_f1 = df[~df.isin(improve_target_f1)].dropna()
mean_not_improve_target_f1 = not_improve_target_f1.mean()
std_not_improve_target_f1 = not_improve_target_f1.std()

print("Average mean metrics in original predicted class for rows that did NOT improve f1 in target class({} samples)".format(len(not_improve_target_f1)))
print("{:<30} {:<15} {:<20}".format("Metric", "Pre-Edit", "Post-Edit"))
for metric in orig_metrics:
    print("{:<30} {:.3f}({:.3f}) {:<6} {:.3f}({:.3f})".format(
        metric, 
        mean_not_improve_target_f1[metric.format("Pre")], 
        std_not_improve_target_f1[metric.format("Pre")], 
        "",
        mean_not_improve_target_f1[metric.format("Post")],
        std_not_improve_target_f1[metric.format("Post")]))

Average mean metrics in original predicted class for rows that improved f1 in target class (6 samples)
Metric                         Pre-Edit        Post-Edit           
{} Orig Pred Precision         0.705(0.064)        0.770(0.074)
{} Orig Pred Recall            0.509(0.132)        0.449(0.164)
{} Orig Pred F1                0.584(0.095)        0.550(0.122)
Average mean metrics in original predicted class for rows that did NOT improve f1 in target class(122 samples)
Metric                         Pre-Edit        Post-Edit           
{} Orig Pred Precision         0.680(0.074)        0.460(0.413)
{} Orig Pred Recall            0.639(0.137)        0.248(0.267)
{} Orig Pred F1                0.650(0.091)        0.298(0.306)


In [188]:
metric_name = 'Recall'
pos_target_pos_orig = df[ 
    (df['Post Target {}'.format(metric_name)] > df['Pre Target {}'.format(metric_name)]) &
    (df['Post Orig Pred {}'.format(metric_name)] > df['Pre Orig Pred {}'.format(metric_name)])]  

pos_target_neg_orig = df[ 
    (df['Post Target {}'.format(metric_name)] > df['Pre Target {}'.format(metric_name)]) &
    (df['Post Orig Pred {}'.format(metric_name)] < df['Pre Orig Pred {}'.format(metric_name)])] 

neg_target_pos_orig = df[ 
    (df['Post Target {}'.format(metric_name)] < df['Pre Target {}'.format(metric_name)]) &
    (df['Post Orig Pred {}'.format(metric_name)] > df['Pre Orig Pred {}'.format(metric_name)])]  

neg_target_neg_orig = df[ 
    (df['Post Target {}'.format(metric_name)] < df['Pre Target {}'.format(metric_name)]) &
    (df['Post Orig Pred {}'.format(metric_name)] < df['Pre Orig Pred {}'.format(metric_name)])]  


In [189]:
print("Metric: {}".format(metric_name))
print("Pos Target Pos Orig: {}".format(len(pos_target_pos_orig)))
print("Pos Target Neg Orig: {}".format(len(pos_target_neg_orig)))
print("Neg Target Pos Orig: {}".format(len(neg_target_pos_orig)))
print("Neg Target Neg Orig: {}".format(len(neg_target_neg_orig)))

Metric: Recall
Pos Target Pos Orig: 7
Pos Target Neg Orig: 120
Neg Target Pos Orig: 0
Neg Target Neg Orig: 1


### Deeper understanding in how those that increase both target and original class predictions differ from those that decrease original class recall

In [197]:
metric_name = 'Recall'
pos_target_pos_orig = df[ 
    (df['Post Target {}'.format(metric_name)] > df['Pre Target {}'.format(metric_name)]) &
    (df['Post Orig Pred {}'.format(metric_name)] > df['Pre Orig Pred {}'.format(metric_name)])]  

pos_target_neg_orig = df[ 
    (df['Post Target {}'.format(metric_name)] > df['Pre Target {}'.format(metric_name)]) &
    (df['Post Orig Pred {}'.format(metric_name)] < df['Pre Orig Pred {}'.format(metric_name)])] 



In [201]:
def get_unique(df_):
    ids = df_['ID']
    print(ids.iloc[0])
    unique_key_images = {}
    unique_segmentation_methods = {}
    unique_modification_methods = {}
    for id_ in ids:
        split_id = id_.split('/')
        key_id = split_id[0]
        if key_id in unique_key_images:
            unique_key_images[key_id] += 1
        else: 
            unique_key_images[key_id] = 1
        
        val_id = split_id[1].split('_')
        segmentation_method = val_id[0]
        if segmentation_method in unique_segmentation_methods:
            unique_segmentation_methods[segmentation_method] += 1
        else:
            unique_segmentation_methods[segmentation_method] = 1
        
        modification_method = val_id[1]
        if modification_method in unique_modification_methods:
            unique_modification_methods[modification_method] += 1
        else:
            unique_modification_methods[modification_method] = 1
    print(unique_key_images)
    print(unique_segmentation_methods)
    print(unique_modification_methods)
    
        
print("Increase target and orig. prediction")
print_summary(pos_target_pos_orig)
get_unique(pos_target_pos_orig)
print(pos_target_pos_orig['ID'])

print("Increase target and decrease orig. prediction")
print_summary(pos_target_neg_orig)
get_unique(pos_target_neg_orig)

Increase target and orig. prediction
Metric                         Pre-Edit        Post-Edit           
{} Accuracy                    0.687           0.673(0.010)
{} Mean Precision              0.692           0.708(0.013)
{} Mean Recall                 0.687           0.673(0.010)
{} Mean F1                     0.684           0.675(0.006)

{} Target Precision            0.554           0.430(0.068)
{} Target Recall               0.549           0.684(0.094)
{} Target F1                   0.551           0.519(0.027)

{} Orig Pred Precision         0.703           0.696(0.052)
{} Orig Pred Recall            0.626           0.648(0.104)
{} Orig Pred F1                0.658           0.668(0.077)

cat-train-n02128757_4479/quickshift_masked_0
{'cat-train-n02128757_4479': 1, 'cat-train-n02128925_4745': 5, 'cat-train-n01322898_6579': 1}
{'quickshift': 3, 'felzenszwalb': 2, 'watershed': 1, 'slic': 1}
{'masked': 5, 'gaussian': 2}
7         cat-train-n02128757_4479/quickshift_masked_0
19   

## Hypothesis 4: If a gaussian noise segment is producing sucessful change, will the masked segment as well?

Result: not necessarily

In [171]:
# Count number of rows that are gaussian noise and are masked
masked_rows = df[df['ID'].str.contains('masked')]
n_masked = len(masked_rows)

gaussian_rows = df[df['ID'].str.contains('gaussian')]
n_gaussian = len(gaussian_rows)
print("{} masked modifications\n{} Gaussian modifications".format(n_masked, n_gaussian))

# For gaussian rows, are their corresponding masked segment also there?
gaussian_IDs = gaussian_rows['ID']
corresponding_masked_IDs = gaussian_IDs.replace('gaussian', 'masked', regex=True)
# print(corresponding_masked_IDs)

segments_with_both_gaussian_and_masked = list(set(corresponding_masked_IDs) & set(df['ID']))
n_both = len(segments_with_both_gaussian_and_masked)
print("{}/{} gaussian modifications have corresponding masked segment as success:".format(n_both, n_gaussian))
# print(segments_with_both_gaussian_and_masked)

98 masked modifications
30 Gaussian modifications
14/30 gaussian modifications have corresponding masked segment as success:


## Hypothesis 5: Smaller segments will produce smaller changes


## Hypothesis 6: The neighbors of the value images should be less affected than the neighbors of the key images

Compare distance between key-keyN and val-valN before and after the edit. The difference should be smaller for val-valN

In [ ]:
# mean_diff_key_keyN = (df['Post key-keyN (F)'] - df['Pre key-keyN (F)']).mean()
mean_df = df.mean()
for data_type in ["F", "L"]:
    mean_pre_key_keyN = mean_df['Pre key-keyN ({})'.format(data_type)]
    mean_post_key_keyN = mean_df['Post key-keyN ({})'.format(data_type)]
    diff_key_keyN = mean_post_key_keyN - mean_pre_key_keyN
    percent_diff_key_keyN = diff_key_keyN / mean_pre_key_keyN * 100
    # mean_diff_val_valN = (df['Post val-valN (F)'] - df['Pre val-valN (F)']).mean()
    mean_pre_val_valN = mean_df['Pre val-valN ({})'.format(data_type)]
    mean_post_val_valN = mean_df['Post val-valN ({})'.format(data_type)]
    diff_val_valN = mean_post_val_valN - mean_pre_val_valN
    percent_diff_val_valN = diff_val_valN / mean_pre_val_valN * 100

    print("{}".format("Features" if data_type == 'F' else "Logits"))
    print("\tKey -> Key Neighbors: {:.3f} ==> {:.3f}".format(mean_pre_key_keyN, mean_post_key_keyN))
    print("\t\tMean difference: {:.3f} ({:.2f}%)".format(diff_key_keyN, percent_diff_key_keyN))
    print("\tVal -> Val Neighbors: {:.3f} ==> {:.3f}".format(mean_pre_val_valN, mean_post_val_valN))
    print("\t\tMean difference: {:.3f} ({:.2f}%)".format(diff_val_valN, percent_diff_val_valN))

## Hypothesis 6: Edits that shared an original image will have smaller spread in post edit metrics


In [157]:
# Obtain unique key images
ids = list(df['ID'])
unique_keys = set()
for image_id in ids:
    key_id = image_id.split('/')[0]
    unique_keys.add(key_id)
    
metrics = [['{} Accuracy', '{} Mean Precision', '{} Mean Recall', '{} Mean F1'], 
           ['{} Target Precision', '{} Target Recall', '{} Target F1'],
           ['{} Orig Pred Precision', '{} Orig Pred Recall', '{} Orig Pred F1'],
           ['{} key-val (F)']]


key_image_stds = pd.DataFrame()
small_spread_keys = []
for unique_key in unique_keys:
    cur_rows = df[df['ID'].str.contains(unique_key)]
    mean_cur_rows = cur_rows.mean()
    std_cur_rows = cur_rows.std()
    # print("Accuracy STD: {:.3f}".format(std_cur_rows['Post Accuracy']))
    # print("Precision STD: {:.3f}".format(std_cur_rows['Post Mean Precision']))
    # print("Recall STD: {:.3f}".format(std_cur_rows['Post Mean Recall']))
    if std_cur_rows['Post Accuracy'] < 0.01:
        small_spread_keys.append(unique_key)
    else:
        print(cur_rows['Post Accuracy'])
    if len(cur_rows) > 1:
        key_image_stds = key_image_stds.append(std_cur_rows, ignore_index=True)
        

mean_key_image_stds = key_image_stds.mean()
print("{:<30} {:<15} {:<20}".format("Metric", "STD Overall", "Avg STD Grouped by Key Image"))
for row in metrics:
    for metric in row:
        print("{:<30} {:<15.3f} {:.3f}".format(
            metric.format("Post"), std_df[metric.format("Post")],
            mean_key_image_stds[metric.format("Post")]))
    print("")

# print("Average accuracy standard deviation for an image: {}".format(key_image_stds['Post Accuracy']))
print(small_spread_keys)

16    0.687371
17    0.688529
18    0.669086
19    0.620000
20    0.685500
21    0.685386
22    0.620614
23    0.687300
Name: Post Accuracy, dtype: float64
14    0.641871
Name: Post Accuracy, dtype: float64
36    0.672886
Name: Post Accuracy, dtype: float64
15    0.675343
Name: Post Accuracy, dtype: float64
24    0.687429
25    0.636157
26    0.687714
27    0.685071
28    0.688229
Name: Post Accuracy, dtype: float64
6    0.668386
Name: Post Accuracy, dtype: float64
Metric                         STD Overall     Avg STD Grouped by Key Image
Post Accuracy                  0.018           0.014
Post Mean Precision            0.032           0.019
Post Mean Recall               0.018           0.014
Post Mean F1                   0.025           0.019

Post Target Precision          0.107           0.088
Post Target Recall             0.049           0.044
Post Target F1                 0.036           0.025

Post Orig Pred Precision       0.264           0.178
Post Orig Pred Recall       

## Hypothesis 9: Edits that share a original prediction will have more similar post edit metric results

In [139]:
unique_original_predictions = set(df['Pre key Prediction'])
print(unique_original_predictions)

for og_prediction in unique_original_predictions:
    cur_rows = df[df['Pre key Prediction'] == og_prediction]
    # print(len(cur_rows))
    mean_cur_rows = cur_rows.mean()
    std_cur_rows = cur_rows.std()
    print(std_cur_rows['Post Accuracy'])

{2, 3, 4, 6, 7}
0.004919442891969285
0.006148371534599434
0.024926112445238086
0.006064829466900532
0.028172920852970768
